In [ ]:
# Import the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.preprocessing import StandardScaler

# Data Cleaning, Missing Data and Normalization

## Removing Non-Predictive Features

In [ ]:
# Load the data
df_unclean = pd.read_csv('CommViolPredUnnormalizedData.csv', na_values=['?'])

In [ ]:
# Removing the non-predictive features
columns_to_remove = [
    'communityname',
    'state',
    'countyCode',
    'communityCode',
    'fold',
    'Unnamed: 0'
]

# Removing potential target features
columns_to_remove += [
        'murders', 
        'murdPerPop', 
        'rapes', 
        'rapesPerPop', 
        'robberies', 
        'robbbPerPop', 
        'assaults', 
        'burglaries',
        'burglPerPop', 
        'larcenies', 
        'larcPerPop',
        'autoTheft', 
        'autoTheftPerPop', 
        'arsons', 
        'arsonsPerPop',
        'ViolentCrimesPerPop', 
        'nonViolPerPop'
    ]


df_cleaned = df_unclean.drop(columns=columns_to_remove)

## Handling Features with Missing Data

First, the missing data in the dataset is identified. For each feature  with misisng data, the percentage of missing data is calculated to help decide on how to handle the missing data.

In [ ]:

# Calculate missing values statistics
total_cells = np.prod(df_cleaned.shape)
total_missing = df_cleaned.isnull().sum().sum()
        

missing_values = df_cleaned.isnull().sum()
missing_percentages = (missing_values / len(df_cleaned)) * 100
        
# Create summary DataFrame
missing_info = pd.DataFrame({'Missing Values': missing_values, 'Percentage Missing': missing_percentages})
missing_info = missing_info[missing_info['Missing Values'] > 0].sort_values('Percentage Missing', ascending=False)

# Display missing values statistics
if not missing_info.empty:
    print("-" * 50)
    for idx, row in missing_info.iterrows():
        print(f"{idx}:")
        print(f"  Missing values: {row['Missing Values']:,}")
        print(f"  Percentage missing: {row['Percentage Missing']:.2f}%")
        print("-" * 50)
else:
    print("\nNo missing values found in the dataset!")

The features which have 84.51% missing data are removed from the dataset. Removing these features may lead to a loss of information. However, imputing the data when the percentage of missing values is high makes it difficult to reliably impute the missing data and could lead to bias.

In [ ]:
# Drop rows with high percentage of missing values
for idx, row in missing_info.iterrows():
    if row['Percentage Missing'] > 50:
        df_cleaned = df_cleaned.drop(columns=idx)

For the target the `assaultPerPop` feature, the rows with missing data are removed from the dataset since there are a small number of rows with missing values. Additonally, since this is the target feature, imputing the missing values could lead to biased results.

In [ ]:
# Drop rows with missing values
df_cleaned = df_cleaned.dropna(subset=['assaultPerPop'])

For the `OtherPerCap` feature, the missing values are imputed with the median of the feature as there is only one missing value. The median is chosen as there are outliers in the feature and a large standard deviation. Moreover, the difference in the mean and median suggests that the data is skewed.

In [ ]:
# Statistics for OtherPerCap
print(df_cleaned['OtherPerCap'].describe())

# Calculate the lower and upper bounds for outliers
Q1 = df_cleaned['OtherPerCap'].quantile(0.25)
Q3 = df_cleaned['OtherPerCap'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Find outliers
outliers = df_cleaned[(df_cleaned['OtherPerCap'] < lower_bound) | 
                      (df_cleaned['OtherPerCap'] > upper_bound)]['OtherPerCap']

print(f"Number of outliers: {len(outliers)}")

In [ ]:
# Impute missing values with the median
df_cleaned.loc[:, 'OtherPerCap'] = df_cleaned['OtherPerCap'].fillna(df_cleaned['OtherPerCap'].median())

## Normalizing the Data 

In [ ]:
df_cleaned.hist(bins=30, figsize=(40, 35))
plt.show()

As can be seen from the above historgrams, a lot of the features contain outliers. Hence, a standard scaler is chosen to normalize the data since it is less sensitive to outliers. A standard scaler changes the data into standard scores which results in the mean of the feature being 0 and the standard deviation being 1.

In [ ]:
# import minmaxscaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
df_normalized = scaler.fit_transform(df_cleaned)

# Filter Methods

In [ ]:

# Convert df_normalized back to DataFrame
df_normalized_df = pd.DataFrame(df_normalized, columns=df_cleaned.columns)

# Create correlation matrix
correlation_matrix = df_normalized_df.corr()

# Set up the matplotlib figure
plt.figure(figsize=(12, 10))

# Create heatmap using seaborn
heatmap = sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', fmt='.2f', linewidths=0.5)         # Make the plot square-shaped

plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Set correlation threshold
CORRELATION_THRESHOLD = 0.7  # High correlation threshold

# Find highly correlated feature pairs
highly_correlated = np.where(np.abs(correlation_matrix) > CORRELATION_THRESHOLD)
highly_correlated = [(correlation_matrix.index[x], correlation_matrix.columns[y], correlation_matrix.iloc[x, y]) 
                    for x, y in zip(*highly_correlated) if x != y and x < y]  # Remove self-correlations and duplicates

print("Highly correlated feature pairs (correlation > 0.7):")
for feat1, feat2, corr in highly_correlated:
    print(f"{feat1} -- {feat2}: {corr:.3f}")

# Keep one feature from each highly correlated pair
features_to_drop = set()
for feat1, feat2, corr in highly_correlated:
    # Keep the feature that has higher correlation with target (assaultPerPop)
    corr1 = abs(correlation_matrix.loc[feat1, 'assaultPerPop'])
    corr2 = abs(correlation_matrix.loc[feat2, 'assaultPerPop'])
    if corr1 < corr2:
        features_to_drop.add(feat1)
    else:
        features_to_drop.add(feat2)

# Convert df_normalized back to DataFrame
df_normalized_df = pd.DataFrame(df_normalized, columns=df_cleaned.columns)

# Drop highly correlated features
df_normalized_uncorrelated = df_normalized_df.drop(columns=features_to_drop)
print(f"\nRemoved {len(features_to_drop)} features")
print("Remaining features:", list(df_normalized_uncorrelated.columns))

In [ ]:
from sklearn.model_selection import train_test_split

# Split features and target
X = df_normalized_uncorrelated.drop('assaultPerPop', axis=1)
y = df_normalized_uncorrelated['assaultPerPop']

# Split data with 80-20 ratio and fixed random state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=46
)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

# Initialize the MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=46)

# Train the model
mlp.fit(X_train, y_train)

# Predict on the test set
y_pred = mlp.predict(X_test)

# Compute the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.10f}")

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.plot(mlp.loss_curve_)
plt.title('MLPRegressor Learning Curve')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.grid(True)
plt.tight_layout()
plt.show()

## Task 3

In [ ]:
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

def sequential_forward_selection(X, y, max_features=40, n_splits=5, random_state=46):
    # Initialize variables
    n_features = X.shape[1]
    remaining_features = list(X.columns)
    selected_features = []
    feature_scores = {}
    
    # Create K-fold cross validator
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    while len(selected_features) < max_features and len(remaining_features) > 0:
        best_score = float('inf')
        best_feature = None
        
        # Try each remaining feature
        for feature in remaining_features:
            current_features = selected_features + [feature]
            scores = []
            
            # Perform k-fold cross validation
            for train_idx, val_idx in kf.split(X):
                # Split data
                X_train, X_val = X.iloc[train_idx][current_features], X.iloc[val_idx][current_features]
                y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
                
                # Train model
                model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=random_state)
                model.fit(X_train, y_train)
                
                # Make predictions and calculate MSE
                y_pred = model.predict(X_val)
                mse = mean_squared_error(y_val, y_pred)
                scores.append(mse)
            
            # Calculate average MSE across folds
            avg_score = np.mean(scores)
            
            # Update best feature if current feature performs better
            if avg_score < best_score:
                best_score = avg_score
                best_feature = feature
        
        # Check if adding the best feature improves the score
        if len(feature_scores) > 0 and best_score >= list(feature_scores.values())[-1]:
            print("No improvement in MSE. Stopping early.")
            break
            
        # Add best feature to selected features
        if best_feature is not None:
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            feature_scores[len(selected_features)] = best_score
            
            print(f"Selected feature {len(selected_features)}: {best_feature} (MSE: {best_score:.10f})")
    
    return selected_features, feature_scores

# Run SFS on the normalized data
X = df_normalized_uncorrelated.drop('assaultPerPop', axis=1)
y = df_normalized_uncorrelated['assaultPerPop']

selected_features, feature_scores = sequential_forward_selection(X, y)

# Plot the feature selection results
plt.figure(figsize=(12, 6))
plt.plot(list(feature_scores.keys()), list(feature_scores.values()), marker='o')
plt.xlabel('Number of Features')
plt.ylabel('Mean Squared Error')
plt.title('Sequential Forward Selection Results')
plt.grid(True)
plt.tight_layout()
plt.show()

# Train final model with selected features
X_selected = X[selected_features]
X_train_sfs, X_test_sfs, y_train, y_test = train_test_split(
    X_selected, y,
    test_size=0.2,
    random_state=46
)

mlp_sfs = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=46)
mlp_sfs.fit(X_train_sfs, y_train)

# Predict and calculate MSE
y_pred_sfs = mlp_sfs.predict(X_test_sfs)
mse_sfs = mean_squared_error(y_test, y_pred_sfs)
print(f"\nFinal MSE with SFS selected features: {mse_sfs:.10f}")


df_normalized_uncorrelated.to_csv('normalized_data.csv', index=False)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

def sequential_backward_selection(X, y, min_features=40, n_splits=5, random_state=46):
    # Initialize variables
    n_features = X.shape[1]
    remaining_features = list(X.columns)
    feature_scores = {n_features: float('inf')}  # Start with worst possible score
    
    # Create K-fold cross validator
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    # Get initial score with all features
    scores = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=random_state)
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)
        scores.append(mse)
    
    feature_scores[len(remaining_features)] = np.mean(scores)
    print(f"Initial MSE with all {len(remaining_features)} features: {feature_scores[len(remaining_features)]:.10f}")
    
    # Continue removing features until we reach min_features
    while len(remaining_features) > min_features:
        best_score = float('inf')
        worst_feature = None
        
        # Try removing each remaining feature
        for feature in remaining_features:
            current_features = [f for f in remaining_features if f != feature]
            scores = []
            
            # Perform k-fold cross validation
            for train_idx, val_idx in kf.split(X):
                X_train, X_val = X.iloc[train_idx][current_features], X.iloc[val_idx][current_features]
                y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
                
                model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=random_state)
                model.fit(X_train, y_train)
                
                y_pred = model.predict(X_val)
                mse = mean_squared_error(y_val, y_pred)
                scores.append(mse)
            
            # Calculate average MSE across folds
            avg_score = np.mean(scores)
            
            # Update worst feature if current removal gives better performance
            if avg_score < best_score:
                best_score = avg_score
                worst_feature = feature
        
        # Check if removing the worst feature improves the score
        if best_score >= feature_scores[len(remaining_features)]:
            print("No improvement in MSE. Stopping early.")
            break
            
        # Remove worst feature
        if worst_feature is not None:
            remaining_features.remove(worst_feature)
            feature_scores[len(remaining_features)] = best_score
            print(f"Removed feature (remaining: {len(remaining_features)}): {worst_feature} (MSE: {best_score:.10f})")
    
    return remaining_features, feature_scores

# Run SBS on the normalized data
X = df_normalized_uncorrelated.drop('assaultPerPop', axis=1)
y = df_normalized_uncorrelated['assaultPerPop']

selected_features, feature_scores = sequential_backward_selection(X, y)

# Plot the feature selection results
plt.figure(figsize=(12, 6))
plt.plot(list(feature_scores.keys()), list(feature_scores.values()), marker='o')
plt.xlabel('Number of Features')
plt.ylabel('Mean Squared Error')
plt.title('Sequential Backward Selection Results')
plt.grid(True)
plt.tight_layout()
plt.show()

# Train final model with selected features
X_selected = X[selected_features]
X_train_sbs, X_test_sbs, y_train, y_test = train_test_split(
    X_selected, y,
    test_size=0.2,
    random_state=46
)

mlp_sbs = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=46)
mlp_sbs.fit(X_train_sbs, y_train)

# Predict and calculate MSE
y_pred_sbs = mlp_sbs.predict(X_test_sbs)
mse_sbs = mean_squared_error(y_test, y_pred_sbs)
print(f"\nFinal MSE with SBS selected features: {mse_sbs:.10f}")